In [10]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm

# --- 1. Descargar todas las legislaturas ---
url_legislaturas = "https://opendata.congreso.cl/wscamaradiputados.asmx/getLegislaturas"
response = requests.get(url_legislaturas)
response.raise_for_status()

root = ET.fromstring(response.content)

legislaturas = []
for leg in root.findall(".//{http://tempuri.org/}Legislatura"):
    legislaturas.append({
        "id_legislatura": leg.findtext("{http://tempuri.org/}ID"),
        "numero_legislatura": leg.findtext("{http://tempuri.org/}Numero"),
        "tipo_legislatura": leg.find("{http://tempuri.org/}Tipo").text,
        "codigo_tipo": leg.find("{http://tempuri.org/}Tipo").attrib.get("Codigo"),
        "fecha_inicio": leg.findtext("{http://tempuri.org/}FechaInicio"),
        "fecha_termino": leg.findtext("{http://tempuri.org/}FechaTermino")
    })

df_leg = pd.DataFrame(legislaturas)

print(f"✅ Legislaturas descargadas: {len(df_leg)}")

# --- 2. Descargar todas las sesiones por legislatura ---
all_sesiones = []

for _, row in tqdm(df_leg.iterrows(), total=len(df_leg), desc="Descargando sesiones"):
    leg_id = row["id_legislatura"]
    url_sesiones = f"https://opendata.congreso.cl/wscamaradiputados.asmx/getSesiones?prmLegislaturaID={leg_id}"
    
    try:
        res = requests.get(url_sesiones)
        res.raise_for_status()
        root_ses = ET.fromstring(res.content)
        
        for ses in root_ses.findall(".//{http://tempuri.org/}Sesion"):
            all_sesiones.append({
                "id_legislatura": leg_id,
                "numero_legislatura": row["numero_legislatura"],
                "id_sesion": ses.findtext("{http://tempuri.org/}ID"),
                "numero_sesion": ses.findtext("{http://tempuri.org/}Numero"),
                "fecha_inicio": ses.findtext("{http://tempuri.org/}Fecha"),
                "fecha_termino": ses.findtext("{http://tempuri.org/}FechaTermino"),
                "tipo_sesion": ses.find("{http://tempuri.org/}Tipo").text if ses.find("{http://tempuri.org/}Tipo") is not None else None,
                "codigo_tipo_sesion": ses.find("{http://tempuri.org/}Tipo").attrib.get("Codigo") if ses.find("{http://tempuri.org/}Tipo") is not None else None,
                "estado_sesion": ses.find("{http://tempuri.org/}Estado").text if ses.find("{http://tempuri.org/}Estado") is not None else None,
                "codigo_estado": ses.find("{http://tempuri.org/}Estado").attrib.get("Codigo") if ses.find("{http://tempuri.org/}Estado") is not None else None
            })
    
    except Exception as e:
        print(f"⚠️ Error al descargar sesiones para legislatura {leg_id}: {e}")

# --- 3. Crear DataFrame final ---
df_sesiones = pd.DataFrame(all_sesiones)

print(f"✅ Sesiones totales descargadas: {len(df_sesiones)}")
display(df_sesiones.head())


✅ Legislaturas descargadas: 55


Descargando sesiones: 100%|██████████| 55/55 [00:06<00:00,  8.06it/s]

✅ Sesiones totales descargadas: 4411


,id_legislatura,numero_legislatura,id_sesion,numero_sesion,fecha_inicio,fecha_termino,tipo_sesion,codigo_tipo_sesion,estado_sesion,codigo_estado
0,3,319,850,2,1990-03-20T15:01:00,1990-03-20T15:02:00,Ordinaria,60,Celebrada,1
1,3,319,851,3,1990-03-21T00:00:00,1990-03-21T00:00:00,Ordinaria,60,Celebrada,1
2,3,319,852,5,1990-03-28T00:00:00,1990-03-28T00:00:00,Ordinaria,60,Celebrada,1
3,3,319,853,6,1990-04-03T15:00:00,1990-04-03T15:00:00,Ordinaria,60,Celebrada,1
4,3,319,854,7,1990-04-04T00:00:00,1990-04-04T00:00:00,Ordinaria,60,Celebrada,1


In [14]:
import requests
import xml.etree.ElementTree as ET

# --- ID de sesión que quieres probar ---
sesion_id = 4714  # cámbialo por cualquier ID

# --- URL base del boletín ---
url_boletin = f"https://opendata.congreso.cl/wscamaradiputados.asmx/getSesionBoletinXML?prmSesionID={sesion_id}"

print(f"📥 Descargando boletín XML para sesión {sesion_id}...\n")

try:
    response = requests.get(url_boletin)
    response.raise_for_status()
    content = response.content.decode("utf-8", errors="replace").strip()

    # --- Intentar parsear el XML ---
    try:
        root = ET.fromstring(content)
        print(f"✅ XML bien formado para sesión {sesion_id}")
        print(f"Etiqueta raíz: {root.tag}")

    except ET.ParseError as e:
        print(f"❌ Error al parsear XML para sesión {sesion_id}: {e}")
        print("\n--- Contenido recibido ---\n")
        print(content[:1500])  # imprime los primeros 1500 caracteres
        print("\n--- Fin del contenido ---")

except Exception as e:
    print(f"⚠️ Error al descargar o abrir el XML para sesión {sesion_id}: {e}")


📥 Descargando boletín XML para sesión 4714...

❌ Error al parsear XML para sesión 4714: no element found: line 1, column 38

--- Contenido recibido ---

<?xml version="1.0" encoding="utf-8"?>

--- Fin del contenido ---


In [16]:
df_sesiones

,id_legislatura,numero_legislatura,id_sesion,numero_sesion,fecha_inicio,fecha_termino,tipo_sesion,codigo_tipo_sesion,estado_sesion,codigo_estado
0,3,319,850,2,1990-03-20T15:01:00,1990-03-20T15:02:00,Ordinaria,60,Celebrada,1
1,3,319,851,3,1990-03-21T00:00:00,1990-03-21T00:00:00,Ordinaria,60,Celebrada,1
2,3,319,852,5,1990-03-28T00:00:00,1990-03-28T00:00:00,Ordinaria,60,Celebrada,1
3,3,319,853,6,1990-04-03T15:00:00,1990-04-03T15:00:00,Ordinaria,60,Celebrada,1
4,3,319,854,7,1990-04-04T00:00:00,1990-04-04T00:00:00,Ordinaria,60,Celebrada,1
...,...,...,...,...,...,...,...,...,...,...
4406,57,373,4672,43,2025-07-01T10:05:41,2025-07-01T14:38:46,Ordinaria,60,Celebrada,1
4407,57,373,4673,44,2025-07-02T10:03:28,2025-07-02T14:05:33,Ordinaria,60,Celebrada,1
4408,57,373,4676,46,2025-07-08T10:05:16,2025-07-08T13:36:20,Ordinaria,60,Celebrada,1
4409,57,373,4677,47,2025-07-09T10:06:07,2025-07-09T14:12:41,Ordinaria,60,Celebrada,1


In [ ]:
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import xmltodict
import json
from datetime import datetime
from time import sleep

# -------------------------------
# Configuración de URLs y carpetas
# -------------------------------
URL_LEGISLATURAS = "https://opendata.congreso.cl/wscamaradiputados.asmx/getLegislaturas"
URL_SESIONES = "https://opendata.congreso.cl/wscamaradiputados.asmx/getSesiones?prmLegislaturaID={}"
URL_BOLETIN = "https://opendata.congreso.cl/wscamaradiputados.asmx/getSesionBoletinXML?prmSesionID={}"

base_dir = "boletines_json"
os.makedirs(base_dir, exist_ok=True)

# -------------------------------
# Función para limpiar nodos innecesarios
# -------------------------------
def clean_dict(data):
    """Elimina claves 'br' vacías o con solo None, recursivamente."""
    if isinstance(data, dict):
        cleaned = {}
        for k, v in data.items():
            if k == "br":
                # Omitir si es None, vacío o solo contiene None
                if v is None:
                    continue
                if isinstance(v, list) and all(x is None for x in v):
                    continue
            cleaned[k] = clean_dict(v)
        return cleaned
    elif isinstance(data, list):
        return [clean_dict(x) for x in data if x is not None]
    else:
        return data

# -------------------------------
# 1. Descargar lista de legislaturas
# -------------------------------
print("📘 Descargando lista de legislaturas...")
resp = requests.get(URL_LEGISLATURAS)
resp.raise_for_status()
root = ET.fromstring(resp.content)

legislaturas = []
for leg in root.findall(".//{http://tempuri.org/}Legislatura"):
    lid = int(leg.find("{http://tempuri.org/}ID").text)
    num = leg.find("{http://tempuri.org/}Numero").text
    tipo = leg.find("{http://tempuri.org/}Tipo").attrib.get("Codigo")
    legislaturas.append({"legislatura_id": lid, "numero": num, "tipo": tipo})

print(f"✅ Se encontraron {len(legislaturas)} legislaturas.\n")

# -------------------------------
# 2. Descargar sesiones y boletines válidos
# -------------------------------
records = []

for leg in legislaturas:
    lid = leg["legislatura_id"]
    numero_leg = leg["numero"]
    folder = os.path.join(base_dir, f"legislatura_{numero_leg}")
    os.makedirs(folder, exist_ok=True)

    print(f"📄 Descargando sesiones de legislatura {numero_leg} (ID {lid})...")

    try:
        res_ses = requests.get(URL_SESIONES.format(lid), timeout=20)
        res_ses.raise_for_status()
        root_ses = ET.fromstring(res_ses.content)
        sesiones = root_ses.findall(".//{http://tempuri.org/}Sesion")

        print(f"📌 {len(sesiones)} sesiones encontradas.\n")

        for ses in sesiones:
            sid = int(ses.find("{http://tempuri.org/}ID").text)
            num_sesion = ses.find("{http://tempuri.org/}Numero").text
            fecha = ses.find("{http://tempuri.org/}Fecha").text
            fecha_ter = ses.find("{http://tempuri.org/}FechaTermino").text
            tipo = ses.find("{http://tempuri.org/}Tipo").attrib.get("Codigo")
            estado = ses.find("{http://tempuri.org/}Estado").attrib.get("Codigo")

            print(f"📥 Verificando boletín XML para sesión {sid}...")

            try:
                res_boletin = requests.get(URL_BOLETIN.format(sid), timeout=25)
                res_boletin.raise_for_status()
                content = res_boletin.content.decode("utf-8", errors="replace").strip()

                # Si el contenido es demasiado corto, se descarta
                if len(content) < 100:
                    print(f"⚪ Boletín {sid} vacío, se omite.\n")
                    continue

                # Intentar parsear el XML
                try:
                    xml_parsed = xmltodict.parse(content)
                    estado_xml = "válido"
                except Exception:
                    print(f"⚠️ Error de formato en boletín {sid}, intentando reparar...")
                    try:
                        content_fixed = content.replace("&", "&amp;")
                        xml_parsed = xmltodict.parse(content_fixed)
                        estado_xml = "reparado"
                    except Exception:
                        print(f"❌ No se pudo reparar boletín {sid}.\n")
                        continue

                # Limpiar los nodos "br"
                xml_cleaned = clean_dict(xml_parsed)

                # Guardar como JSON con nombre por número de sesión
                json_path = os.path.join(folder, f"boletin_{num_sesion}.json")
                with open(json_path, "w", encoding="utf-8") as f:
                    json.dump(xml_cleaned, f, ensure_ascii=False, indent=2)

                print(f"✅ Guardado boletín sesión {num_sesion} ({estado_xml})\n")

                records.append({
                    "legislatura_id": lid,
                    "legislatura_numero": numero_leg,
                    "sesion_id": sid,
                    "numero_sesion": num_sesion,
                    "fecha_inicio": fecha,
                    "fecha_termino": fecha_ter,
                    "tipo_sesion": tipo,
                    "estado_sesion": estado,
                    "estado_xml": estado_xml,
                    "ruta_json": json_path,
                    "bytes": len(content.encode("utf-8")),
                    "fecha_descarga": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })

            except Exception as e:
                print(f"❌ Error descargando boletín sesión {sid}: {e}\n")

            sleep(0.5)

    except Exception as e:
        print(f"⚠️ Error procesando legislatura {numero_leg}: {e}\n")

# -------------------------------
# 3. Guardar dataset final
# -------------------------------
df = pd.DataFrame(records)
csv_path = os.path.join(base_dir, "dataset_boletines.csv")
df.to_csv(csv_path, index=False, encoding="utf-8")

print("\n✅ Descarga completa.")
print(f"📁 Archivos JSON guardados en: {os.path.abspath(base_dir)}")
print(f"🧾 Dataset total: {len(df)} boletines descargados.")
df.head()


📘 Descargando lista de legislaturas...
✅ Se encontraron 55 legislaturas.

📄 Descargando sesiones de legislatura 319 (ID 3)...
📌 22 sesiones encontradas.

📥 Verificando boletín XML para sesión 850...
⚪ Boletín 850 vacío, se omite.

📥 Verificando boletín XML para sesión 851...
⚪ Boletín 851 vacío, se omite.

📥 Verificando boletín XML para sesión 852...
⚪ Boletín 852 vacío, se omite.

📥 Verificando boletín XML para sesión 853...
⚪ Boletín 853 vacío, se omite.

📥 Verificando boletín XML para sesión 854...
⚪ Boletín 854 vacío, se omite.

📥 Verificando boletín XML para sesión 855...
⚪ Boletín 855 vacío, se omite.

📥 Verificando boletín XML para sesión 856...
⚪ Boletín 856 vacío, se omite.

📥 Verificando boletín XML para sesión 857...
⚪ Boletín 857 vacío, se omite.

📥 Verificando boletín XML para sesión 858...
⚪ Boletín 858 vacío, se omite.

📥 Verificando boletín XML para sesión 859...
⚪ Boletín 859 vacío, se omite.

📥 Verificando boletín XML para sesión 860...
⚪ Boletín 860 vacío, se omite.


In [1]:
!pip install pymupdf

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import fitz  # PyMuPDF
import json
import os
import re
from datetime import datetime
import locale # Para manejar meses en español

# --- Configuración ---
directorio_pdf_entrada = "C:/Users/conjv/Desktop/pdfs_descargados"  # CAMBIA ESTO a tu carpeta con PDFs
directorio_json_salida = "C:/Users/conjv/Desktop/json_descargados" # CAMBIA ESTO a tu carpeta de salida deseada

# Configurar locale a español para reconocer los nombres de los meses
# Puede variar según tu sistema operativo ('es_ES.UTF-8', 'es_CL.UTF-8', 'Spanish_Spain.1252' en Windows)
try:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8') 
except locale.Error:
    try:
        locale.setlocale(locale.LC_TIME, 'Spanish') # Para Windows
    except locale.Error:
        print("Advertencia: No se pudo configurar el locale a español. La extracción de fechas puede fallar.")
        # Continuar con el locale por defecto si falla español

def extraer_metadatos_desde_nombre(filename):
    """
    Intenta extraer metadatos del nombre del archivo PDF.
    Formato esperado: NUMLEG__LEGISLATURA__*_NUMSESION__TIPO__*FECHA*.pdf
    Ejemplo: 370__LEGISLATURA__mar._2022_-_mar._2023__5___ordinaria__23_marzo___2022.pdf
    """
    metadata = {
        '@NUMERO': '0',
        '@FECHA_INICIO': 'Unknown',
        '@FECHA_TERMINO': 'Unknown',
        '@TIPO': 'Unknown',
        'legislatura': 'Unknown'
    }

    # Extraer Legislatura (ej: 370)
    match_leg = re.match(r'(\d+)__LEGISLATURA', filename, re.IGNORECASE)
    if match_leg:
        metadata['legislatura'] = match_leg.group(1)

    # Buscar Número de Sesión y Tipo (ej: __5___ordinaria__)
    # Buscamos __NUMERO___TIPO__ (con underscores variables)
    match_ses_tipo = re.search(r'__(\d+)_{3,}(ordinaria|especial|extraordinaria)_{2,}', filename, re.IGNORECASE)
    if match_ses_tipo:
        metadata['@NUMERO'] = match_ses_tipo.group(1)
        metadata['@TIPO'] = match_ses_tipo.group(2).capitalize()

    # Buscar Fecha (ej: __23_marzo___2022) - Asume día_mes___año cerca del final
    # Este patrón es delicado y puede necesitar ajustes
    match_fecha = re.search(r'__(\d{1,2})_([a-zA-Záéíóúüñ]+)_{3,}(\d{4})\.pdf$', filename, re.IGNORECASE)
    if match_fecha:
        dia, mes_str, anio = match_fecha.groups()
        try:
            # Intentar convertir mes en español a número
            fecha_obj = datetime.strptime(f"{dia} {mes_str} {anio}", '%d %B %Y')
            fecha_formateada = fecha_obj.strftime('%d-%m-%Y')
            metadata['@FECHA_INICIO'] = fecha_formateada
            metadata['@FECHA_TERMINO'] = fecha_formateada # Asumir misma fecha
        except ValueError:
            # Si falla la conversión (locale incorrecto o mes mal escrito)
            print(f"Advertencia: No se pudo convertir la fecha '{dia} {mes_str} {anio}' para {filename}")
            metadata['@FECHA_INICIO'] = f"{dia}-{mes_str}-{anio}" # Guardar como se encontró
            metadata['@FECHA_TERMINO'] = metadata['@FECHA_INICIO']

    # Si no se encontró el tipo antes, buscarlo de forma más simple
    if metadata['@TIPO'] == 'Unknown':
         if 'ordinaria' in filename.lower():
             metadata['@TIPO'] = 'Ordinaria'
         elif 'especial' in filename.lower():
              metadata['@TIPO'] = 'Especial'
         elif 'extraordinaria' in filename.lower():
             metadata['@TIPO'] = 'Extraordinaria'


    return metadata

def convertir_pdf_a_json_boletin_v2(ruta_pdf, ruta_json_salida):
    """
    Convierte un PDF de boletín a un JSON con estructura básica,
    extrayendo metadatos del nombre del archivo.
    """
    try:
        # Extraer metadatos del nombre del archivo ANTES de abrir el PDF
        filename = os.path.basename(ruta_pdf)
        metadatos = extraer_metadatos_desde_nombre(filename)

        # Extraer texto del PDF
        doc = fitz.open(ruta_pdf)
        texto_completo = ""
        for page in doc:
            texto_completo += page.get_text("text") + "\n--- PAGE BREAK ---\n" # Marcar saltos de página
        doc.close()

        # Crear la estructura JSON
        json_data = {
            "BOLETINXML": {
                # Usar @TEMPLATE genérico si no se extrajo el tipo
                "@TEMPLATE": f"Sesión {metadatos['@TIPO']}" if metadatos['@TIPO'] != 'Unknown' else "Sesión", 
                "@VALID": "False", # Asumir False por defecto
                "SESION": {
                    "@NUMERO": metadatos['@NUMERO'],
                    "@FECHA_INICIO": metadatos['@FECHA_INICIO'],
                    "@FECHA_TERMINO": metadatos['@FECHA_TERMINO'], # Asumido igual a inicio
                    "@TIPO": metadatos['@TIPO'],
                    # Incluir metadato extraído de legislatura si se desea
                    # "@LEGISLATURA": metadatos['legislatura'], 
                    "#text": texto_completo.strip() 
                }
            }
        }

        # Asegurar que el directorio de salida exista
        os.makedirs(os.path.dirname(ruta_json_salida), exist_ok=True)

        # Guardar el JSON
        with open(ruta_json_salida, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=2)

        print(f"✅ Convertido: {filename} -> {os.path.basename(ruta_json_salida)}")
        return True

    except Exception as e:
        print(f"❌ Error convirtiendo {os.path.basename(ruta_pdf)}: {e}")
        return False

# --- Procesamiento Principal ---
print(f"🚀 Iniciando conversión de PDFs en '{directorio_pdf_entrada}'...")
archivos_convertidos = 0
archivos_fallidos = 0

# Crear directorio base de salida si no existe
os.makedirs(directorio_json_salida, exist_ok=True)

# Listar archivos en la carpeta de entrada
try:
    lista_pdfs = [f for f in os.listdir(directorio_pdf_entrada) if f.lower().endswith(".pdf")]
except FileNotFoundError:
     print(f"❌ Error: La carpeta de entrada '{directorio_pdf_entrada}' no existe.")
     lista_pdfs = [] # Detener si no existe la carpeta

if not lista_pdfs:
    print("🤷 No se encontraron archivos PDF en la carpeta de entrada.")
else:
    print(f"🔍 Encontrados {len(lista_pdfs)} archivos PDF para procesar.")
    for filename in lista_pdfs:
        ruta_pdf = os.path.join(directorio_pdf_entrada, filename)
        
        # Crear nombre de archivo JSON de salida
        nombre_json = os.path.splitext(filename)[0] + ".json"
        ruta_json = os.path.join(directorio_json_salida, nombre_json)

        if convertir_pdf_a_json_boletin_v2(ruta_pdf, ruta_json):
            archivos_convertidos += 1
        else:
            archivos_fallidos += 1
            
# --- Resumen Final ---
print("\n🏁 Proceso de conversión finalizado.")
print(f"   Archivos convertidos exitosamente: {archivos_convertidos}")
print(f"   Archivos con errores: {archivos_fallidos}")
if archivos_convertidos > 0:
    print(f"   Resultados guardados en: {os.path.abspath(directorio_json_salida)}")

🚀 Iniciando conversión de PDFs en 'C:/Users/conjv/Desktop/pdfs_descargados'...
🔍 Encontrados 478 archivos PDF para procesar.
✅ Convertido: 370__LEGISLATURA__mar._2022_-_mar._2023__0___congreso_pleno__11_marzo___2022.pdf -> 370__LEGISLATURA__mar._2022_-_mar._2023__0___congreso_pleno__11_marzo___2022.json
✅ Convertido: 370__LEGISLATURA__mar._2022_-_mar._2023__100___ordinaria__22_noviembre___2022.pdf -> 370__LEGISLATURA__mar._2022_-_mar._2023__100___ordinaria__22_noviembre___2022.json
✅ Convertido: 370__LEGISLATURA__mar._2022_-_mar._2023__101___ordinaria__23_noviembre___2022.pdf -> 370__LEGISLATURA__mar._2022_-_mar._2023__101___ordinaria__23_noviembre___2022.json
✅ Convertido: 370__LEGISLATURA__mar._2022_-_mar._2023__102___ordinaria__28_noviembre___2022.pdf -> 370__LEGISLATURA__mar._2022_-_mar._2023__102___ordinaria__28_noviembre___2022.json
✅ Convertido: 370__LEGISLATURA__mar._2022_-_mar._2023__103___ordinaria__29_noviembre___2022.pdf -> 370__LEGISLATURA__mar._2022_-_mar._2023__103___ord

In [5]:
import os
import re

# --- Configuración ---
# Directorio base donde están tus carpetas 'legislatura_XXX'
# (Basado en tu ejemplo: json_descargados/legislatura_372/...)
base_dir = "json_descargados" 
# ---------------------

# Patrón Regex para buscar el número de sesión en el *nombre del archivo*
# Esto es más fiable que el @NUMERO_SESION dentro del JSON,
# que a veces es '0' (como en boletin_111.json).
#
# Este patrón busca: __[NUMERO]___[TIPO]__
# Ejemplo: ...__5___ordinaria__... o ...__1___ordinaria__...
#
# __(\d+)_ -> Captura el grupo de dígitos (el número de sesión)
# _{2,}     -> Busca al menos 2 guiones bajos (para ___)
# (ordinaria|especial|extraordinaria) -> Busca el tipo de sesión para confirmar
#
filename_pattern = re.compile(r'__(\d+)_{2,}(ordinaria|especial|extraordinaria)', re.IGNORECASE)

print(f"🚀 Iniciando el proceso de renombrado en: {os.path.abspath(base_dir)}")
print("   Modo: Extrayendo N° de Sesión desde el *nombre del archivo* (es el método más fiable).")

files_renamed = 0
files_failed = 0
files_skipped = 0

if not os.path.isdir(base_dir):
    print(f"❌ Error: El directorio base '{base_dir}' no existe. Por favor, verifica la ruta.")
else:
    # Usamos os.walk para recorrer todas las subcarpetas
    for root, dirs, files in os.walk(base_dir):
        
        # No procesamos el directorio base, solo sus subcarpetas
        # (Asumiendo que los JSON están en legislatura_XXX)
        if root == base_dir:
            continue
            
        print(f"\n📁 Revisando carpeta: {root}")
        
        for filename in files:
            # Solo procesar archivos .json
            if filename.endswith(".json"):
                original_file_path = os.path.join(root, filename)

                # Omitir archivos que ya están renombrados
                if filename.startswith("boletin_"):
                    files_skipped += 1
                    continue

                # 1. Buscar el patrón en el nombre del archivo
                match = filename_pattern.search(filename)
                
                if match:
                    try:
                        # 2. Extraer el número de sesión (el primer grupo capturado)
                        sesion_numero_str = match.group(1)
                        # Limpiar ceros a la izquierda (ej: "005" -> "5")
                        sesion_numero_limpio = str(int(sesion_numero_str)) 
                        
                        # 3. Construir el nuevo nombre y ruta
                        nuevo_nombre = f"boletin_{sesion_numero_limpio}.json"
                        nuevo_file_path = os.path.join(root, nuevo_nombre)
                        
                        if os.path.exists(nuevo_file_path):
                            print(f"  ⚠️ Advertencia: Ya existe {nuevo_nombre}. Se eliminará el original duplicado: {filename}")
                            os.remove(original_file_path) # Elimina el archivo largo si el 'boletin_X.json' ya existe
                            files_failed += 1
                        else:
                            # 4. Renombrar el archivo
                            os.rename(original_file_path, nuevo_file_path)
                            print(f"  ✅ Renombrado: {filename} -> {nuevo_nombre}")
                            files_renamed += 1
                    
                    except Exception as e:
                        print(f"  ❌ Error (Inesperado) procesando {filename}: {e}")
                        files_failed += 1
                else:
                    # Si el nombre del archivo NO coincide con el patrón esperado
                    print(f"  ❌ Error (Patrón no encontrado): {filename}")
                    files_failed += 1

print("\n🏁 Proceso finalizado.")
print(f"   Archivos renombrados: {files_renamed}")
print(f"   Archivos omitidos (ya nombrados): {files_skipped}")
print(f"   Archivos fallidos/duplicados: {files_failed}")

🚀 Iniciando el proceso de renombrado en: C:\Users\conjv\Desktop\json_descargados
   Modo: Extrayendo N° de Sesión desde el *nombre del archivo* (es el método más fiable).

📁 Revisando carpeta: json_descargados\legislatura_372
  ✅ Renombrado: 372__LEGISLATURA__mar._2024_-_mar._2025__100___especial__12_noviembre___2024.json -> boletin_100.json
  ✅ Renombrado: 372__LEGISLATURA__mar._2024_-_mar._2025__101___especial__12_noviembre___2024.json -> boletin_101.json
  ✅ Renombrado: 372__LEGISLATURA__mar._2024_-_mar._2025__102___ordinaria__13_noviembre___2024.json -> boletin_102.json
  ✅ Renombrado: 372__LEGISLATURA__mar._2024_-_mar._2025__103___especial__14_noviembre___2024.json -> boletin_103.json
  ✅ Renombrado: 372__LEGISLATURA__mar._2024_-_mar._2025__104___especial__15_noviembre___2024.json -> boletin_104.json
  ✅ Renombrado: 372__LEGISLATURA__mar._2024_-_mar._2025__105___ordinaria__18_noviembre___2024.json -> boletin_105.json
  ✅ Renombrado: 372__LEGISLATURA__mar._2024_-_mar._2025__106___o

In [266]:
import json
import pandas as pd
import os
import re
import traceback

# --- Funciones Auxiliares (Robustecidas v14) ---

def safe_strip(data, default=""):
    """Limpia espacios en blanco de forma segura si es un string, sino devuelve default."""
    if isinstance(data, str):
        # Reemplazar múltiples espacios/saltos de línea con un solo
        return re.sub(r'\s+', ' ', data).strip()
    return default

def extract_legislatura(portada_text):
    """(NUEVO v13) Extrae el número de legislatura desde el texto de portada."""
    if not isinstance(portada_text, str):
        return 'N/A'
    
    # Busca "LEGISLATURA 370ª" o "LEGISLATURA 370a" o "Legislatura 360ª"
    match = re.search(r'LEGISLATURA\s+(\d+)[ªa]', portada_text, re.IGNORECASE)
    if match:
        return match.group(1)
        
    return 'N/A'

def parse_speaker_from_cue(cue_text, attribute_name=""):
    """
    (v12) Extrae nombre COMPLETO y tipo desde el CUE del texto.
    (v14) Mejorado para capturar nombres en formato APELLIDO (don Nombre) y 'La señorita'.
    """
    speaker_name = "Desconocido"
    speaker_type = "Desconocido"
    role_from_cue = ""
    role_text_lower = "" # (NUEVO v14) Inicializar

    # 1. Extraer Tipo (Rol)
    role_match = re.search(r'\((.*?)\)', cue_text)
    is_name_in_parens = False # (NUEVO v14)
    
    if role_match:
        role_from_cue = role_match.group(1).strip()
        role_text_lower = role_from_cue.lower()
        
        # (MODIFICADO v14) Si el paréntesis contiene "don" o "doña", NO es un rol, es parte del nombre.
        if 'don ' in role_text_lower or 'doña ' in role_text_lower:
            is_name_in_parens = True

    # (MODIFICADO v14) Asignar tipo basado en si el paréntesis era un nombre o un rol
    if role_match and not is_name_in_parens:
        # Mapeo de roles (si el paréntesis NO era un nombre)
        if 'vicepresidente' in role_text_lower: speaker_type = "Vicepresidente"
        elif 'presidente accidental' in role_text_lower: speaker_type = "Presidente (Accidental)"
        elif 'presidente' in role_text_lower: speaker_type = "Presidente"
        elif 'ministro' in role_text_lower or 'ministra' in role_text_lower: speaker_type = "Ministro"
        elif 'subsecretario' in role_text_lower or 'subsecretaria' in role_text_lower: speaker_type = "Subsecretario"
        elif 'secretario' in role_text_lower: speaker_type = "Secretario"
        elif 'prosecretario' in role_text_lower: speaker_type = "Prosecretario"
        elif 'senador' in role_text_lower or 'senadora' in role_text_lower: speaker_type = "Senador"
        elif any(role in role_text_lower for role in ['autoridad', 'director', 'fiscal', 'contralor', 'defensor']): speaker_type = "Autoridad"
        else: speaker_type = "Diputado" # Default
    else:
        # Heurística (si no hay paréntesis, o si el paréntesis era un nombre)
        lower_cue = cue_text.lower()
        # Evitar que "don" o "doña" se confundan con roles
        if not is_name_in_parens:
            if 'secretario' in lower_cue: speaker_type = "Secretario"
            elif 'prosecretario' in lower_cue: speaker_type = "Prosecretario"
            elif 'ministro' in lower_cue: speaker_type = "Ministro"
            else: speaker_type = "Diputado"
        else:
            speaker_type = "Diputado"

    # 2. Extraer Nombre Completo
    # (MODIFICADO v14) Añadido 'La señorita'
    name_match_regex = r'(?:El señor|La señora|La señorita)\s+(.*?)(?=\s*(?:\(.*\)\.-|\.-))'
    name_match = re.search(name_match_regex, cue_text, re.DOTALL)
    
    if name_match:
        full_name_match = name_match.group(1).strip()
        # (MODIFICADO v14) No quitar 'don' o 'doña' aún.
        speaker_name_cleaned = re.sub(r'\s+', ' ', full_name_match.replace('.', '')).strip()

        if speaker_name_cleaned.upper() == role_from_cue.upper() or \
           (speaker_name_cleaned.upper() == speaker_type.upper() and speaker_type not in ["Diputado", "Desconocido"]):
            
            if attribute_name and attribute_name != "Desconocido" and attribute_name.upper() != speaker_type.upper():
                   speaker_name = attribute_name
            else:
                   speaker_name = speaker_type
        else:
            # (NUEVO v14) Lógica para combinar APELLIDO (don Nombre)
            if is_name_in_parens and (speaker_type == "Diputado" or speaker_name == "Desconocido"):
                
                # speaker_name_cleaned es el APELLIDO (ej. RAMÍREZ)
                # role_from_cue es el Nombre (ej. don Matías)
                first_name_in_parens = re.sub(r'(don |doña )', '', role_from_cue, flags=re.IGNORECASE).strip()
                
                # Formatear a "Nombre Apellido" (ej. "Matías RAMÍREZ")
                speaker_name = f"{first_name_in_parens} {speaker_name_cleaned}"
            else:
                # Limpieza final si no es el caso APELLIDO (don Nombre)
                # (ej. "MUÑOZ, doña Adriana" o "LUCK")
                speaker_name = speaker_name_cleaned.replace('don ', '').replace('doña ', '').strip()
    else:
       if speaker_type != "Diputado": speaker_name = speaker_type
       elif attribute_name and attribute_name != "Desconocido": speaker_name = attribute_name

    # (NUEVO v14) Refinamiento final: si el nombre es solo un APELLIDO (mayúsculas)
    # e 'attribute_name' es un nombre completo, usar 'attribute_name'.
    # (ej. cue es "El señor DITTBORN.-" y attribute_name es "Dittborn, Julio")
    if (speaker_name.isupper() and ' ' not in speaker_name and 
        attribute_name and attribute_name != "Desconocido" and 
        speaker_name.upper() in attribute_name.upper().split(',')[0].strip()): # Comparamos solo con el apellido
        
        speaker_name = attribute_name
    
    speaker_name = speaker_name.rstrip(',').strip()
    return speaker_name, speaker_type


def segment_text_block(text_block, base_speaker_name="Texto No Estructurado", base_speaker_type="Narrador"):
    """
    (NUEVO v13) Divide un bloque #text en múltiples turnos (Cue + Texto).
    También detecta títulos de sección en texto plano.
    """
    dialogues = []
    
    # Regex para cues de orador. Captura (Grupo 1: El cue completo)
    speaker_regex_pattern = r'((?:El señor|La señora|La señorita)\s+.*?\.-\s*)' # (MODIFICADO v14) Añadido 'La señorita'
    # Regex para títulos de sección (Materia)
    section_title_regex_pattern = r'(\n\s*(?:[IVX]+\.?|A?)\s*.-?\s*[A-ZÁÉÍÓÚÑÑ\s,]{5,}(?=\n\s*\n|\n\s*(?:El señor|La señora|La señorita)))' # (MODIFICADO v14) Añadido 'La señorita'
    
    # Patrón combinado para buscar Cues O Títulos
    combined_regex = re.compile(f'({speaker_regex_pattern}|{section_title_regex_pattern})', re.DOTALL)

    # Usar finditer para obtener las posiciones de inicio/fin de cues y títulos
    matches = list(combined_regex.finditer(text_block))
    
    current_topic_title = "Texto Principal No Estructurado" # Default para fallback
    current_speaker_name = base_speaker_name
    current_speaker_type = base_speaker_type
    
    start_index = 0
    if matches:
        first_match_start = matches[0].start()
        # 1. Capturar texto ANTES del primer cue/título
        if first_match_start > 0:
            initial_text = text_block[0:first_match_start].strip()
            if initial_text:
                dialogues.append({
                    'name': current_speaker_name,
                    'type': current_speaker_type,
                    'text': safe_strip(initial_text),
                    'materia_fallback': current_topic_title # (FIX v12) Pasar el título
                })
        start_index = matches[0].end()
    else:
        # No hay matches, todo el bloque es un solo diálogo
        full_text = text_block.strip()
        if full_text:
            dialogues.append({
                'name': current_speaker_name,
                'type': current_speaker_type,
                'text': safe_strip(full_text),
                'materia_fallback': current_topic_title
            })
        return dialogues # Salir si no hay matches

    # 2. Iterar sobre los matches encontrados
    for i, match in enumerate(matches):
        # Determinar si este match es un CUE o un TÍTULO
        is_cue = re.fullmatch(speaker_regex_pattern, match.group(1), re.DOTALL)
        is_title = re.fullmatch(section_title_regex_pattern, match.group(1), re.DOTALL)

        # Determinar el texto que sigue a este match
        end_index = matches[i+1].start() if (i + 1) < len(matches) else len(text_block)
        following_text = text_block[match.end():end_index].strip()
        
        if is_cue:
            cue_text = match.group(1).strip()
            # Parsear el orador desde el cue
            current_speaker_name, current_speaker_type = parse_speaker_from_cue(cue_text, base_speaker_name)
            
            # (FIX v13) Crear UNA sola fila por intervención (Cue + Texto)
            full_dialogue_text = f"{cue_text} {following_text}".strip()
            
            dialogues.append({
                'name': current_speaker_name,
                'type': current_speaker_type,
                'text': safe_strip(full_dialogue_text), # Guardar el texto completo
                'materia_fallback': current_topic_title
            })
        
        elif is_title:
            # Es un título de sección (Materia)
            title_text = safe_strip(match.group(1))
            current_topic_title = re.sub(r'^[IVX]+\.?\s*.-?\s*', '', title_text).strip() # Limpiar "V. -"
            current_topic_title = re.sub(r'\s+', ' ', current_topic_title) # Normalizar espacios
            
            # El texto que sigue (si existe) es un 'Narrador' bajo este nuevo título
            if following_text:
                dialogues.append({
                    'name': "Narrador",
                    'type': "Narrador",
                    'text': safe_strip(following_text),
                    'materia_fallback': current_topic_title
                })
            # El próximo cue pertenecerá a este nuevo título

    return dialogues


def parse_speaker_info_from_attributes(intervention_key, intervention_data):
    """(v11) Extrae nombre COMPLETO y tipo del orador base desde atributos JSON."""
    speaker_name = "Desconocido"
    speaker_type = "Desconocido"
    raw_info = None

    try:
        # Presidente / Vicepresidente
        if intervention_key in ['INTERVENCION_PRESIDENTE', 'INTERVENCION_1VICE']:
            speaker_type = "Presidente" if intervention_key == 'INTERVENCION_PRESIDENTE' else "Vicepresidente"
            raw_info = intervention_data.get('@DIPUTADO', speaker_type)
            raw_info_str = safe_strip(raw_info, "")
            if raw_info_str:
                speaker_name_cleaned = re.sub(r'\s*\([^)]*\)$', '', raw_info_str).strip()
                speaker_name_cleaned = re.sub(r'^(El señor|La señora|La señorita)\s*', '', speaker_name_cleaned, flags=re.IGNORECASE).strip() # (MODIFICADO v14)
                speaker_name_cleaned = speaker_name_cleaned.replace('don ', '').replace('doña ', '').strip()
                if 'Vicepresident' in raw_info_str or 'Vicepresidenta' in raw_info_str: speaker_type = "Vicepresidente"
                elif 'accidental' in raw_info_str.lower() and 'Presidente' in speaker_type: speaker_type = "Presidente (Accidental)"
                speaker_name = speaker_name_cleaned if speaker_name_cleaned else speaker_type
            else: speaker_name = speaker_type
        # Diputado
        elif intervention_key == 'INTERVENCION_DIPUTADO':
            speaker_type = "Diputado"
            raw_info = intervention_data.get('@DIPUTADO', 'Diputado Desconocido')
            raw_info_str = safe_strip(raw_info, "")
            if raw_info_str:
                speaker_name_cleaned = re.sub(r'\s*\([^)]*\)$', '', raw_info_str).strip()
                speaker_name_cleaned = speaker_name_cleaned.replace('don ', '').replace('doña ', '').strip()
                speaker_name = speaker_name_cleaned if speaker_name_cleaned else "Diputado Desconocido"
            else: speaker_name = "Diputado Desconocido"
        # Autoridad
        elif intervention_key == 'INTERVENCION_AUTORIDAD':
            speaker_type = "Autoridad"
            raw_info = intervention_data.get('@DESCRIPCION', 'Autoridad Desconocida')
            raw_info_str = safe_strip(raw_info, "")
            if raw_info_str:
                name_match = re.search(r"(?:señor|señora)\s+([A-ZÁÉÍÓÚÑÑ][A-Za-záéíóúñÁÉÍÓÚÑÑ\s,'-]+)(?:,|\.|$)", raw_info_str)
                if name_match: speaker_name = name_match.group(1).strip()
                else:
                    speaker_name = raw_info_str.split(',')[0].strip();
                    if len(speaker_name) > 60: speaker_name = "Autoridad (ver desc.)"
                lower_info = raw_info_str.lower()
                if 'ministro' in lower_info or 'ministra' in lower_info: speaker_type = "Ministro"
                elif 'subsecretario' in lower_info or 'subsecretaria' in lower_info: speaker_type = "Subsecretario"
                elif 'senador' in lower_info or 'senadora' in lower_info: speaker_type = "Senador"
                elif 'fiscal' in lower_info: speaker_type = "Fiscal"
                elif 'director' in lower_info or 'directora' in lower_info : speaker_type = "Director"
                elif 'contralor' in lower_info: speaker_type = "Contralor"
                elif 'defensor' in lower_info: speaker_type = "Defensor"
                else: speaker_type = "Autoridad"
            else: speaker_name = "Autoridad Desconocida"
        # Otros roles
        elif intervention_key == 'INTERVENCION_SECRETARIO':
            speaker_type = "Secretario"
            raw_info = intervention_data.get('@SECRETARIO', 'Secretario')
            raw_info_str = safe_strip(raw_info, "")
            if raw_info_str and raw_info_str != "Secretario":
                 speaker_name_cleaned = re.sub(r'\s*\([^)]*\)$', '', raw_info_str).strip()
                 speaker_name_cleaned = re.sub(r'^(El señor|La señora|La señorita)\s*', '', speaker_name_cleaned, flags=re.IGNORECASE).strip() # (MODIFICADO v14)
                 speaker_name_cleaned = speaker_name_cleaned.replace('don ', '').replace('doña ', '').strip()
                 speaker_name = speaker_name_cleaned if speaker_name_cleaned else "Secretario"
            else: speaker_name = "Secretario"
        elif intervention_key == 'INTERVENCION_PROSECRETARIO':
            speaker_type = "Prosecretario"
            raw_info = intervention_data.get('@PROSECRETARIO', 'Prosecretario')
            raw_info_str = safe_strip(raw_info, "")
            if raw_info_str and raw_info_str != "Prosecretario":
                 speaker_name_cleaned = re.sub(r'\s*\([^)]*\)$', '', raw_info_str).strip()
                 speaker_name_cleaned = re.sub(r'^(El señor|La señora|La señorita)\s*', '', speaker_name_cleaned, flags=re.IGNORECASE).strip() # (MODIFICADO v14)
                 speaker_name_cleaned = speaker_name_cleaned.replace('don ', '').replace('doña ', '').strip()
                 speaker_name = speaker_name_cleaned if speaker_name_cleaned else "Prosecretario"
            else: speaker_name = "Prosecretario"
        elif intervention_key == 'INTERVENCION_SENADOR':
             speaker_type = "Senador"
             raw_info = intervention_data.get('@SENADOR', 'Senador Desconocido')
             raw_info_str = safe_strip(raw_info, "")
             speaker_name_cleaned = re.sub(r'\s*\([^)]*\)$', '', raw_info_str).strip()
             speaker_name_cleaned = speaker_name_cleaned.replace('don ', '').replace('doña ', '').strip()
             speaker_name = speaker_name_cleaned if speaker_name_cleaned else "Senador Desconocido"
             
        if isinstance(speaker_name, str):
            speaker_name = speaker_name.rstrip(',').strip()
        else: speaker_name = "Error Nombre"

    except Exception as e:
        speaker_name = "Error Nombre Attr"
        speaker_type = "Error Tipo Attr"

    # Devolvemos solo nombre y tipo
    return speaker_name, speaker_type


# Lista global de claves de intervención
potential_intervention_keys = [
    'INTERVENCION_PRESIDENTE', 'INTERVENCION_DIPUTADO', 'INTERVENCION_AUTORIDAD',
    'INTERVENCION_SECRETARIO', 'INTERVENCION_PROSECRETARIO', 'INTERVENCION_1VICE',
    'INTERVENCION_SENADOR', 'INTERVENCION_AFAVOR', 'INTERVENCION_ENCONTRA'
]

# Lista de claves a ignorar en recursión (simplificada)
keys_to_ignore_recursion = [
    '#text', '@TEMPLATE', '@VALID', '@NUMERO', '@FECHA_INICIO', '@FECHA_TERMINO', '@TIPO', 
    'PORTADA', 'INDICE', 'ASISTENCIA', 'FIN_SESION', 
    'DOCUMENTOS_CUENTA', 'OTROS_DOCUMENTOS_CUENTA', 'VOTACION',
    '@DESCRIPCION', '@DIPUTADO', '@DIPUTADOValue', 
    '@SENADOR', '@SENADORValue', '@SECRETARIO', '@SECRETARIOValue', 
    '@PROSECRETARIO', '@PROSECRETARIOValue'
    # No ignoramos TITULO ni @BOLETIN para que se propaguen
]

# --- Función Recursiva (v13 - Lógica de Boletín Corregida) ---
def find_interventions_recursive(item, session_info, current_boletin, file_path, all_dialogues, parent_key=None):
    """
    (Mejorada v13) Recorre recursivamente, propaga el boletín y segmenta #text.
    """
    global potential_intervention_keys, keys_to_ignore_recursion, files_with_unexpected_text

    try:
        # 1. Si es un Diccionario
        if isinstance(item, dict):
            
            # (NUEVO v13) Propagación de Boletín
            # Si este dict define un NUEVO boletín, usarlo. Si no, usar el heredado.
            new_bulletin = item.get('@BOLETIN', current_boletin) # Heredar
            # Si el nuevo boletín es diferente Y no es N/A, actualizar el contexto
            if new_bulletin and new_bulletin != 'N/A' and new_bulletin != current_boletin:
                 current_boletin = new_bulletin # Actualizar
            # Si es N/A, mantener el 'current_boletin' heredado (FIX)
            elif new_bulletin == 'N/A' or new_bulletin is None:
                 new_bulletin = current_boletin

            # --- PASO B: Buscar claves de intervención DENTRO de este dict ---
            for key, value in item.items():
                if key in potential_intervention_keys:
                    
                    if value is None: continue
                    value_list = value if isinstance(value, list) else [value]
                    
                    for intervention_item in value_list:
                        
                        actual_intervention_data = intervention_item
                        actual_key = key

                        if key in ['INTERVENCION_AFAVOR', 'INTERVENCION_ENCONTRA']:
                            if isinstance(intervention_item, dict):
                                nested_keys = [k for k in intervention_item if k.startswith('INTERVENCION_')]
                                if nested_keys:
                                    actual_key = nested_keys[0]; actual_intervention_data = intervention_item[actual_key]
                            else: continue
                        if not isinstance(actual_intervention_data, dict): continue

                        # (NUEVO v13) El boletín es el que heredamos (new_bulletin) o el específico de esta intervención
                        item_bulletin = actual_intervention_data.get('@BOLETIN', new_bulletin)
                        if (item_bulletin == 'N/A' or item_bulletin is None) and new_bulletin != 'N/A':
                             item_bulletin = new_bulletin # Asegurar herencia

                        # --- Extraer Orador Base y Texto ---
                        base_speaker_name, base_speaker_type = parse_speaker_info_from_attributes(actual_key, actual_intervention_data)
                        text_content = actual_intervention_data.get('#text')
                        
                        # --- Segmentar el Texto (NUEVO v13) ---
                        if isinstance(text_content, str):
                            # Usar la nueva lógica de segmentación v13
                            segmented_turns = segment_text_block(text_content, base_speaker_name, base_speaker_type)
                            
                            for turn in segmented_turns:
                                speaker_name_final = turn['name']
                                speaker_type_final = turn['type']
                                
                                # Refinamiento
                                if speaker_name_final == "Desconocido" and base_speaker_name != "Desconocido":
                                    speaker_name_final = base_speaker_name; speaker_type_final = base_speaker_type
                                elif speaker_type_final == "Diputado" and base_speaker_type not in ["Diputado", "Desconocido", "Error Tipo", "Error Tipo Attr"]:
                                     if base_speaker_name and speaker_name_final.upper() in base_speaker_name.upper():
                                          speaker_type_final = base_speaker_type
                                          if len(base_speaker_name) > len(speaker_name_final): speaker_name_final = base_speaker_name
                                if speaker_name_final.upper() == "PRESIDENTE": speaker_type_final = "Presidente"

                                all_dialogues.append({
                                    'numero_legislatura': session_info.get('legislatura', 'N/A'), # (NUEVO v13)
                                    'numero_sesion': session_info.get('id'), # (NUEVO v13)
                                    'fecha_sesion': session_info.get('start_time'), # (NUEVO v13)
                                    'boletin_proyecto': item_bulletin, # (FIX v13)
                                    'speaker_name': speaker_name_final, # <-- CAMBIO: Guardar directo, sin '_raw'
                                    'speaker_type': speaker_type_final,
                                    'text': turn['text'], # Texto segmentado
                                    'source_file': file_path
                                })
                                
                        elif text_content is not None:
                             if file_path not in files_with_unexpected_text: files_with_unexpected_text[file_path] = []
                             files_with_unexpected_text[file_path].append(f"Key: {actual_key}, Type: {type(text_content)}")

            # --- PASO C: Recurrir en todos los hijos ---
            for key, value in item.items():
                if key not in potential_intervention_keys and key not in keys_to_ignore_recursion:
                    # Pasar el boletín actualizado (new_bulletin)
                    find_interventions_recursive(value, session_info, new_bulletin, file_path, all_dialogues, parent_key=key)

        # 2. Si es una Lista
        elif isinstance(item, list):
            for element in item:
                # Pasar el boletín heredado (current_boletin)
                find_interventions_recursive(element, session_info, current_boletin, file_path, all_dialogues, parent_key)
        
        else: return # Caso base

    except Exception as e:
        tb_str = traceback.format_exc()
        if file_path not in error_files:
             error_files.append(f"{file_path} (RecursionError: {e})")
        return


# --- Main Processing Loop (v13) ---
all_dialogues = []
processed_files_count = 0
processed_folders = set()
root_directory = 'C:/Users/conjv/Desktop/boletines_json/'
error_files = []
files_with_unexpected_text = {}
dialogues_per_file = {} # (NUEVO v10)

# (ELIMINADO v13 - Normalización) Carga de nombres canónicos eliminada.

print(f"Starting recursive search in '{root_directory}'...")

for dirpath, dirnames, filenames in os.walk(root_directory):
    dirnames[:] = [d for d in dirnames if not d.startswith('.')]
    
    folder_name = os.path.basename(dirpath)
    if folder_name.startswith('legislatura_'):
        print(f"\nProcessing folder: {folder_name}")
        processed_folders.add(folder_name)
        folder_file_count = 0
        json_files_in_folder = sorted([f for f in filenames if f.startswith('boletin_') and f.endswith('.json')])

        if not json_files_in_folder:
            print(f"  No 'boletin_*.json' files found in this folder.")
            continue

        print(f"  Found {len(json_files_in_folder)} JSON files.")

        for file_name in json_files_in_folder:
            file_path = os.path.join(dirpath, file_name)
            initial_length = len(all_dialogues)
            dialogues_in_this_file = 0

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                session_data = data.get('BOLETINXML', {}).get('SESION', {})
                if not session_data:
                    # (NUEVO v14) Intentar buscar SESION en la raíz si BOLETINXML no existe o está vacío
                    session_data = data.get('SESION', {})
                    if not session_data:
                        print(f"    Skipping {file_name}: No SESION data.")
                        dialogues_per_file[file_path] = 0
                        continue

                # (NUEVO v13) Extraer legislatura
                portada_text = ""
                # (MODIFICADO v14) Buscar PORTADA en session_data
                if isinstance(session_data.get('PORTADA'), dict):
                        portada_text = session_data['PORTADA'].get('#text', "")
                elif isinstance(session_data.get('PORTADA'), str):
                        portada_text = session_data['PORTADA']
                
                legislatura_num = extract_legislatura(portada_text)
                if legislatura_num == 'N/A': # Fallback
                    legislatura_match = re.search(r'legislatura_(\d+)', dirpath, re.IGNORECASE)
                    if legislatura_match: legislatura_num = legislatura_match.group(1)

                session_info = {
                    'id': session_data.get('@NUMERO'),
                    'type': session_data.get('@TIPO'),
                    'start_time': session_data.get('@FECHA_INICIO'),
                    'end_time': session_data.get('@FECHA_TERMINO'),
                    'legislatura': legislatura_num
                }

                # --- Iniciar Búsqueda Estructurada ---
                initial_boletin = 'N/A'
                dialogues_before_structured = len(all_dialogues)
                
                find_interventions_recursive(session_data, session_info, initial_boletin, file_path, all_dialogues)
                
                dialogues_found_structured = len(all_dialogues) - dialogues_before_structured

                # --- (NUEVO v11) Fallback para Texto No Estructurado ---
                unstructured_text = None
                if isinstance(session_data.get('#text'), str):
                    unstructured_text = session_data['#text']

                if dialogues_found_structured < 10 and unstructured_text: # Umbral
                    print(f"    Fallback Regex on {file_name} (found {dialogues_found_structured} structured)...")
                    
                    fallback_turns = segment_text_block(unstructured_text, "Narrador", "Narrador")
                    
                    current_fallback_bulletin = "N/A"
                    
                    for turn in fallback_turns:
                        # (NUEVO v13) Extraer boletín desde la materia detectada en fallback
                        # (FIX v12) Acceder a 'materia_fallback'
                        current_fallback_title = turn.get('materia_fallback', 'Texto Principal No Estructurado')
                        boletin_match = re.search(r'(?:Boletín|Boletines) Nos? ([\d.-]+(?: y [\d.-]+)*)', current_fallback_title, re.IGNORECASE)
                        if boletin_match:
                            current_fallback_bulletin = boletin_match.group(1)
                        # Si no es un título de boletín, mantenemos el N/A
                        elif "ORDEN DEL DÍA" not in current_fallback_title.upper():
                            current_fallback_bulletin = "N/A"
                        
                        all_dialogues.append({
                            'numero_legislatura': session_info.get('legislatura', 'N/A'),
                            'numero_sesion': session_info.get('id'),
                            'fecha_sesion': session_info.get('start_time'),
                            'boletin_proyecto': current_fallback_bulletin,
                            'speaker_name': turn['name'], # <-- CAMBIO: Guardar directo, sin '_raw'
                            'speaker_type': turn['type'],
                            'text': turn['text'],
                            'source_file': file_path
                        })

                # --- Contar y Reportar ---
                processed_files_count += 1
                folder_file_count += 1
                dialogues_in_this_file = len(all_dialogues) - initial_length
                dialogues_per_file[file_path] = dialogues_in_this_file
                # --- (NUEVO v13) Imprimir conteo por archivo ---
                if dialogues_in_this_file > 0:
                    print(f"    Processed {file_name}: Found {dialogues_in_this_file} dialogue turns.")
                else:
                    print(f"    Processed {file_name}: No dialogue turns found.")

            except json.JSONDecodeError:
                print(f"    Error: Could not decode JSON from {file_path}")
                error_files.append(f"{file_path} (JSONDecodeError)")
                dialogues_per_file[file_path] = -1
            except Exception as e:
                tb_str = traceback.format_exc()
                print(f"    An unexpected error occurred processing {file_path}: {e}\n{tb_str[:400]}...")
                error_files.append(f"{file_path} ({type(e).__name__}: {e})")
                dialogues_per_file[file_path] = -1

        print(f"  Finished processing {folder_file_count} files in this folder.")


# --- Post-Procesamiento (ELIMINADO v13): Normalización de Nombres ---
print("\n--- Skipping Post-Processing: Name normalization removed as requested. ---")
df = pd.DataFrame.from_records(all_dialogues)

if df.empty:
     print("\nDataFrame is empty. Skipping Post-Processing.")


# --- Final Output ---
valid_files_count = sum(1 for count in dialogues_per_file.values() if count >= 0)
average_dialogues = 0
if valid_files_count > 0:
    average_dialogues = len(df) / valid_files_count if not df.empty else 0

output_filename = 'segmented_dialogues_all_legislaturas_v14_name_fix.csv'
try:
    if not df.empty:
        # (NUEVO v13) Reordenar y seleccionar columnas finales
        final_columns = [
            'numero_legislatura', 
            'numero_sesion', 
            'fecha_sesion',
            'boletin_proyecto', # Este es el @BOLETIN (ID del proyecto), puede ser N/A
            'speaker_name', # <-- CAMBIO: Esta es ahora la columna "raw" (sin normalizar)
            'speaker_type', 
            'text', 
            'source_file'
        ]
        # Asegurarse de que todas las columnas existan
        for col in final_columns:
            if col not in df.columns:
                df[col] = pd.NA
                
        df = df[final_columns]
        
        # (NUEVO v13) Convertir a tipo string para asegurar compatibilidad
        for col in ['numero_legislatura', 'numero_sesion', 'boletin_proyecto', 'speaker_name', 'speaker_type']:
            if col in df.columns:
                # Rellenar Nulos (NaN, None) con 'N/A' antes de convertir a string
                df[col] = df[col].fillna('N/A').astype(str)

    
        df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        total_folders = len(processed_folders)
        print(f"\nSuccessfully created DataFrame with {len(df)} dialogue turns from {processed_files_count} files ({valid_files_count} successfully processed) across {total_folders} folders.")
        print(f"Average dialogue turns per successfully processed file: {average_dialogues:.2f}")
        print(f"Data saved to {output_filename}")

        if error_files:
            print(f"\nWarning: {len(error_files)} files encountered errors during processing:")
            for i, error_file_info in enumerate(error_files[:30]): print(f"  - {error_file_info}")
            if len(error_files) > 30: print(f"  ... and {len(error_files) - 30} more.")

        if files_with_unexpected_text:
             print(f"\nWarning: Encountered unexpected non-string types for '#text' in {len(files_with_unexpected_text)} files:")
             count = 0
             for file, issues in files_with_unexpected_text.items():
                  print(f"  - {file}: {issues[:3]} {'...' if len(issues) > 3 else ''}")
                  count += 1;
                  if count >= 10: print(f"  ... and {len(files_with_unexpected_text) - 10} more files."); break

        if not df.empty:
            print("\nDataFrame Info:")
            df.info(verbose=True, show_counts=True)
            print("\nDataFrame Head (Muestra Inicial):")
            print(df.head())
            print("\nDataFrame Tail (Muestra Final):")
            print(df.tail())
            
            print("\nVerificación de Boletines (Ejemplos Aleatorios con Boletín):")
            boletin_sample = df[df['boletin_proyecto'] != 'N/A']
            if not boletin_sample.empty:
                print(boletin_sample.sample(min(5, len(boletin_sample)))[['boletin_proyecto', 'speaker_name', 'source_file']])
            else: print("  No se encontraron diálogos con número de boletín.")
                
            print("\nVerificación de Diálogos sin Boletín (N/A):")
            no_boletin_sample = df[df['boletin_proyecto'] == 'N/A']
            if not no_boletin_sample.empty:
                print(no_boletin_sample.sample(min(5, len(no_boletin_sample)))[['boletin_proyecto', 'speaker_name', 'source_file']])
            else: print("  No se encontraron diálogos sin número de boletín.")
                
            print("\nSample Speaker Types Found:")
            print(df['speaker_type'].value_counts().head(20))
            print("\nSample Speaker Names (NUEVO v14 - Corregido):")
            # (NUEVO v14) Mostrar ejemplos de nombres que no sean solo mayúsculas
            fixed_names_sample = df[df['speaker_name'].str.contains(' ', na=False) & ~df['speaker_name'].str.isupper()]
            if not fixed_names_sample.empty:
                print("Ejemplos de nombres combinados (ej. Matías Ramírez):")
                print(fixed_names_sample['speaker_name'].value_counts().head(10))
            else:
                print("No se encontraron nombres combinados (ej. Matías Ramírez) en esta muestra.")
            
            print("\nTop 30 nombres (Raw):")
            print(df['speaker_name'].value_counts().head(30))
            
            print("\nSpeakers with parsing errors (if any):")
            error_name_files = df[df['speaker_name'].isin(['Error Nombre', 'Error Nombre Attr', 'Desconocido'])]
            print(f"  Found {len(error_name_files)} turns with name errors or 'Desconocido'.")
            print(f"  Examples (source files): {error_name_files['source_file'].unique()[:10]}")
        else:
            print("\nWarning: The resulting DataFrame is empty.")

    else:
        print("\nWarning: The resulting DataFrame is empty. No CSV file saved.")
        total_folders = len(processed_folders)
        print(f"Processed {processed_files_count} files ({valid_files_count} successfully processed) across {total_folders} folders, but found 0 dialogue turns.")
        if error_files:
            print(f"\nWarning: {len(error_files)} files encountered errors during processing:")
            for i, error_file_info in enumerate(error_files[:30]): print(f"  - {error_file_info}")


except Exception as e:
    print(f"\nError saving DataFrame to CSV: {e}")

Starting recursive search in 'C:/Users/conjv/Desktop/boletines_json/'...

Processing folder: legislatura_346
  Found 20 JSON files.
    Processed boletin_10.json: Found 146 dialogue turns.
    Processed boletin_11.json: Found 90 dialogue turns.
    Processed boletin_12.json: Found 150 dialogue turns.
    Processed boletin_13.json: Found 148 dialogue turns.
    Processed boletin_15.json: Found 143 dialogue turns.
    Processed boletin_16.json: Found 69 dialogue turns.
    Processed boletin_17.json: Found 90 dialogue turns.
    Processed boletin_18.json: Found 50 dialogue turns.
    Processed boletin_19.json: Found 142 dialogue turns.
    Processed boletin_2.json: Found 23 dialogue turns.
    Processed boletin_20.json: Found 80 dialogue turns.
    Processed boletin_21.json: Found 138 dialogue turns.
    Processed boletin_23.json: Found 64 dialogue turns.
    Processed boletin_24.json: Found 105 dialogue turns.
    Processed boletin_4.json: Found 200 dialogue turns.
    Processed boletin_

In [267]:
df

,numero_legislatura,numero_sesion,fecha_sesion,boletin_proyecto,speaker_name,speaker_type,text,source_file
0,346,10,10-04-2002 10:36:00,2852-07 (S),"MUÑOZ, Adriana",Presidente,"La señora MUÑOZ, doña Adriana (Presidenta).- E...",C:/Users/conjv/Desktop/boletines_json/legislat...
1,346,10,10-04-2002 10:36:00,2852-07 (S),"MUÑOZ, Adriana",Presidente,"La señora MUÑOZ, doña Adriana (Presidenta).- T...",C:/Users/conjv/Desktop/boletines_json/legislat...
2,346,10,10-04-2002 10:36:00,2852-07 (S),"MUÑOZ, Adriana",Presidente,"La señora MUÑOZ, doña Adriana (Presidenta).- L...",C:/Users/conjv/Desktop/boletines_json/legislat...
3,346,10,10-04-2002 10:36:00,2852-07 (S),ESCALONA,Diputado,"El señor ESCALONA.- Muy bien, señora Presidenta.",C:/Users/conjv/Desktop/boletines_json/legislat...
4,346,10,10-04-2002 10:36:00,2852-07 (S),"MUÑOZ, Adriana",Presidente,"La señora MUÑOZ, doña Adriana (Presidenta).- T...",C:/Users/conjv/Desktop/boletines_json/legislat...
...,...,...,...,...,...,...,...,...
253437,373,9,07-04-2025,N/A,RIVAS,Vicepresidente,El señor RIVAS (Vicepresidente).- En discusión...,C:/Users/conjv/Desktop/boletines_json/legislat...
253438,373,9,07-04-2025,N/A,Matías RAMÍREZ,Diputado,El señor RAMÍREZ (don Matías).- Señor Presiden...,C:/Users/conjv/Desktop/boletines_json/legislat...
253439,373,9,07-04-2025,N/A,RIVAS,Vicepresidente,El señor RIVAS (Vicepresidente).- Tiene la pal...,C:/Users/conjv/Desktop/boletines_json/legislat...
253440,373,9,07-04-2025,N/A,GUZMÁN,Diputado,"El señor GUZMÁN.- Señor Presidente, hoy nos co...",C:/Users/conjv/Desktop/boletines_json/legislat...


In [268]:
# --- (NUEVO PASO) Crear un DataFrame filtrado solo con 'Diputado' ---

print("\n--- Creando DataFrame filtrado solo para 'Diputado' ---")

# Filtramos el DataFrame 'df' para quedarnos solo con las filas donde 'speaker_type' es 'Diputado'
# Usamos .copy() para asegurar que sea un DataFrame nuevo e independiente
df_diputados = df[df['speaker_type'] == 'Diputado'].copy()

# Opcional: Mostrar la cabecera del nuevo DataFrame para verificar
print(f"Total de filas en el DF original: {len(df)}")
print(f"Total de filas en el nuevo DF (Solo Diputados): {len(df_diputados)}")
print("\nMuestra del nuevo DataFrame (df_diputados):")
print(df_diputados.head())

# Opcional: Guardar este nuevo DataFrame en su propio CSV
try:
    output_filename_diputados = 'segmented_dialogues_SOLO_DIPUTADOS.csv'
    #df_diputados.to_csv(output_filename_diputados, index=False, encoding='utf-8-sig')
    print(f"\nDataFrame solo con diputados guardado exitosamente en: {output_filename_diputados}")
except Exception as e:
    print(f"\nError al guardar el CSV de solo diputados: {e}")

df_diputados


--- Creando DataFrame filtrado solo para 'Diputado' ---
Total de filas en el DF original: 253442
Total de filas en el nuevo DF (Solo Diputados): 128280

Muestra del nuevo DataFrame (df_diputados):
   numero_legislatura numero_sesion         fecha_sesion boletin_proyecto  \
3                 346            10  10-04-2002 10:36:00      2852-07 (S)   
7                 346            10  10-04-2002 10:36:00      2852-07 (S)   
8                 346            10  10-04-2002 10:36:00      2852-07 (S)   
9                 346            10  10-04-2002 10:36:00      2852-07 (S)   
10                346            10  10-04-2002 10:36:00      2852-07 (S)   

          speaker_name speaker_type  \
3             ESCALONA     Diputado   
7         Bustos, Juan     Diputado   
8       MUÑOZ, Adriana     Diputado   
9   Ortiz, José Miguel     Diputado   
10      MUÑOZ, Adriana     Diputado   

                                                 text  \
3    El señor ESCALONA.- Muy bien, señora Presi

,numero_legislatura,numero_sesion,fecha_sesion,boletin_proyecto,speaker_name,speaker_type,text,source_file
3,346,10,10-04-2002 10:36:00,2852-07 (S),ESCALONA,Diputado,"El señor ESCALONA.- Muy bien, señora Presidenta.",C:/Users/conjv/Desktop/boletines_json/legislat...
7,346,10,10-04-2002 10:36:00,2852-07 (S),"Bustos, Juan",Diputado,"El señor BUSTOS.- Señora Presidenta, la reform...",C:/Users/conjv/Desktop/boletines_json/legislat...
8,346,10,10-04-2002 10:36:00,2852-07 (S),"MUÑOZ, Adriana",Diputado,"La señora MUÑOZ, doña Adriana (Presidenta).- T...",C:/Users/conjv/Desktop/boletines_json/legislat...
9,346,10,10-04-2002 10:36:00,2852-07 (S),"Ortiz, José Miguel",Diputado,"El señor ORTIZ.- Señora Presidenta, el día de ...",C:/Users/conjv/Desktop/boletines_json/legislat...
10,346,10,10-04-2002 10:36:00,2852-07 (S),"MUÑOZ, Adriana",Diputado,"La señora MUÑOZ, doña Adriana (Presidenta).- T...",C:/Users/conjv/Desktop/boletines_json/legislat...
...,...,...,...,...,...,...,...,...
253425,373,9,07-04-2025,N/A,Mónica ARCE,Diputado,La señora ARCE (doña Mónica).- Señor President...,C:/Users/conjv/Desktop/boletines_json/legislat...
253427,373,9,07-04-2025,N/A,KAISER,Diputado,"El señor KAISER.- Señor Presidente, de hecho, ...",C:/Users/conjv/Desktop/boletines_json/legislat...
253436,373,9,07-04-2025,N/A,MATHESON,Diputado,"El señor MATHESON (de pie).- Honorable Cámara,...",C:/Users/conjv/Desktop/boletines_json/legislat...
253438,373,9,07-04-2025,N/A,Matías RAMÍREZ,Diputado,El señor RAMÍREZ (don Matías).- Señor Presiden...,C:/Users/conjv/Desktop/boletines_json/legislat...


In [269]:
import pandas as pd
from rapidfuzz import process, fuzz
import re

# === 1. Cargar datos ===
try:
    nombres_df = pd.read_csv("C:/Users/conjv/Desktop/boletines_json/nombres_unicos.csv")
except FileNotFoundError:
    print("Error: No se pudo encontrar el archivo 'nombres_unicos.csv'. Asegúrate de que la ruta sea correcta.")
    exit()

# === 2. Funciones de limpieza y NUEVAS funciones de género ===

def limpiar_nombre(nombre):
    """Limpia comas, espacios extra y convierte a minúsculas"""
    if pd.isna(nombre):
        return ""
    return nombre.replace(",", "").strip().lower()

def eliminar_ultimo_apellido(nombre):
    """Elimina el último token (apellido final) y convierte a minúsculas"""
    if pd.isna(nombre):
        return ""
    tokens = nombre.strip().split()
    base = " ".join(tokens[:-1]) if len(tokens) > 1 else tokens[0]
    return base.lower()

def guess_gender(full_name):
    """
    (NUEVO) Intenta adivinar el género basado en nombres comunes en español.
    Esto es una heurística, pero es crucial para desambiguar.
    """
    if pd.isna(full_name):
        return 'unknown'
    
    name_lower = full_name.lower()
    tokens = name_lower.split()
    if not tokens:
        return 'unknown'
    
    first_name = tokens[0]
    
    # Lista de nombres femeninos comunes (se puede expandir)
    female_names = ['maría', 'pía', 'adriana', 'mónica', 'karol', 'maya', 'daniella', 'karin', 'carolina', 'maite', 'marisela', 'virginia', 'francesca', 'catalina', 'claudia', 'lorena']
    # Lista de nombres masculinos comunes (se puede expandir)
    male_names = ['jorge', 'camilo', 'juan', 'josé', 'matías', 'sergio', 'christian', 'johannes', 'ramón', 'julio', 'eugenio', 'rené', 'josé', 'camilo', 'germán', 'enrique', 'claudio', 'carlos', 'pablo', 'felipe', 'alfonso', 'antonio']

    if first_name in female_names:
        return 'female'
    if first_name in male_names:
        return 'male'
    
    # Heurística de terminación si no está en la lista
    if first_name.endswith('a'):
        return 'female'
    if first_name.endswith('o'):
        return 'male'
        
    return 'unknown'

def extract_gender_from_cue(text_cue):
    """
    (NUEVO) Extrae el género ("male", "female") del texto de la intervención.
    """
    if pd.isna(text_cue):
        return 'unknown'
    
    text_lower = text_cue.lower()
    
    # Usar regex para buscar las palabras clave al inicio o con paréntesis
    if re.search(r'el señor|\(don ', text_lower):
        return 'male'
    if re.search(r'la señora|la señorita|\(doña ', text_lower):
        return 'female'
        
    return 'unknown'

# === 3. Preparar los DataFrames ===
# Aplicar limpieza
df_diputados["speaker_name_clean"] = df_diputados["speaker_name"].astype(str).apply(limpiar_nombre)
nombres_df["nombre_sin_ultimo"] = nombres_df["nombre_completos"].astype(str).apply(eliminar_ultimo_apellido)

# (NUEVO) Añadir columnas de género a ambos DataFrames
nombres_df["gender_guess"] = nombres_df["nombre_completos"].apply(guess_gender)
df_diputados["cue_gender"] = df_diputados["text"].apply(extract_gender_from_cue)

# (NUEVO) Crear listas de referencia filtradas por género
lista_ref_male = nombres_df[nombres_df["gender_guess"] == 'male']["nombre_sin_ultimo"].tolist()
lista_ref_female = nombres_df[nombres_df["gender_guess"] == 'female']["nombre_sin_ultimo"].tolist()
lista_ref_all = nombres_df["nombre_sin_ultimo"].tolist()

# === 4. Fuzzy matching con rapidfuzz (Función CORREGIDA) ===

def obtener_nombre_completo(row):
    """
    (CORREGIDO) Ahora acepta una 'row' completa para poder usar el género como filtro.
    """
    nombre_limpio = row['speaker_name_clean']
    genero_del_cue = row['cue_gender']
    
    if not nombre_limpio:
        return None
    
    lista_a_usar = lista_ref_all
    
    # (NUEVA LÓGICA) Elegir la lista de búsqueda correcta basada en el género del texto
    if genero_del_cue == 'male':
        lista_a_usar = lista_ref_male
    elif genero_del_cue == 'female':
        lista_a_usar = lista_ref_female
    # Si el género es 'unknown', usará la lista_ref_all (comportamiento anterior)

    match = process.extractOne(
        nombre_limpio,
        lista_a_usar,
        scorer=fuzz.token_set_ratio,
        score_cutoff=85  # Umbral de confianza
    )
    
    if match is None:
        # Si no encontró nada en la lista filtrada, intentar con la lista completa como último recurso
        match = process.extractOne(
            nombre_limpio,
            lista_ref_all,
            scorer=fuzz.token_set_ratio,
            score_cutoff=85
        )
        if match is None:
            return None

    # Encontrar el nombre completo original usando el match
    mejor_match = match[0]
    fila = nombres_df.loc[nombres_df["nombre_sin_ultimo"] == mejor_match, "nombre_completos"]
    return fila.iloc[0] if not fila.empty else None

# === 5. Aplicar la función ===
# (CORREGIDO) Usamos axis=1 para pasar la fila completa a la función
df_diputados["nombre_normalizado_final"] = df_diputados.apply(obtener_nombre_completo, axis=1)

# === 6. Mostrar resultados ===
print("Verificación del match para 'GUZMÁN' (El señor...):")
print(df_diputados[df_diputados['speaker_name'] == 'GUZMÁN'][['text', 'speaker_name_clean', 'cue_gender', 'nombre_normalizado_final']])

print("\nVerificación del match para 'KAISER' (El señor...):")
print(df_diputados[df_diputados['speaker_name'] == 'KAISER'][['text', 'speaker_name_clean', 'cue_gender', 'nombre_normalizado_final']])

print("\nDataFrame final (muestra):")
print(df_diputados[['speaker_name', 'text', 'nombre_normalizado_final']].head())

Verificación del match para 'GUZMÁN' (El señor...):
                                                     text speaker_name_clean  \
195149  El señor GUZMÁN.- Señor Presidente, desde ya a...             guzmán   
196386  El señor GUZMÁN.- Señor Presidente, algunos de...             guzmán   
196432  El señor GUZMÁN.- Señor Presidente, vengo a in...             guzmán   
201328  El señor GUZMÁN.- Señor Presidente, en el mome...             guzmán   
203476  El señor GUZMÁN.- Señora Presidenta, la Conven...             guzmán   
...                                                   ...                ...   
252901  El señor GUZMÁN.- Señor Presidente, nadie debi...             guzmán   
252979  El señor GUZMÁN.- Señor Presidente, el 26 de m...             guzmán   
253269  El señor GUZMÁN.- Señor Presidente, usted tien...             guzmán   
253324  El señor GUZMÁN.- Señor Presidente, por su int...             guzmán   
253440  El señor GUZMÁN.- Señor Presidente, hoy nos co...           

In [274]:
df_diputados

,numero_legislatura,numero_sesion,fecha_sesion,boletin_proyecto,speaker_name,speaker_type,text,source_file,speaker_name_clean,cue_gender,nombre_normalizado_final
3,346,10,10-04-2002 10:36:00,2852-07 (S),ESCALONA,Diputado,"El señor ESCALONA.- Muy bien, señora Presidenta.",C:/Users/conjv/Desktop/boletines_json/legislat...,escalona,male,Camilo Escalona Medina
7,346,10,10-04-2002 10:36:00,2852-07 (S),"Bustos, Juan",Diputado,"El señor BUSTOS.- Señora Presidenta, la reform...",C:/Users/conjv/Desktop/boletines_json/legislat...,bustos juan,male,Juan Bustos Ramírez
8,346,10,10-04-2002 10:36:00,2852-07 (S),"MUÑOZ, Adriana",Diputado,"La señora MUÑOZ, doña Adriana (Presidenta).- T...",C:/Users/conjv/Desktop/boletines_json/legislat...,muñoz adriana,female,Adriana Muñoz D'Albora
9,346,10,10-04-2002 10:36:00,2852-07 (S),"Ortiz, José Miguel",Diputado,"El señor ORTIZ.- Señora Presidenta, el día de ...",C:/Users/conjv/Desktop/boletines_json/legislat...,ortiz josé miguel,male,José Miguel Ortiz Novoa
10,346,10,10-04-2002 10:36:00,2852-07 (S),"MUÑOZ, Adriana",Diputado,"La señora MUÑOZ, doña Adriana (Presidenta).- T...",C:/Users/conjv/Desktop/boletines_json/legislat...,muñoz adriana,female,Adriana Muñoz D'Albora
...,...,...,...,...,...,...,...,...,...,...,...
253425,373,9,07-04-2025,N/A,Mónica ARCE,Diputado,La señora ARCE (doña Mónica).- Señor President...,C:/Users/conjv/Desktop/boletines_json/legislat...,mónica arce,female,Mónica Arce Castro
253427,373,9,07-04-2025,N/A,KAISER,Diputado,"El señor KAISER.- Señor Presidente, de hecho, ...",C:/Users/conjv/Desktop/boletines_json/legislat...,kaiser,male,Johannes Kaiser Barents-Von Hohenhagen
253436,373,9,07-04-2025,N/A,MATHESON,Diputado,"El señor MATHESON (de pie).- Honorable Cámara,...",C:/Users/conjv/Desktop/boletines_json/legislat...,matheson,male,Christian Matheson Villán
253438,373,9,07-04-2025,N/A,Matías RAMÍREZ,Diputado,El señor RAMÍREZ (don Matías).- Señor Presiden...,C:/Users/conjv/Desktop/boletines_json/legislat...,matías ramírez,male,Matías Ramírez Pascal


In [280]:
columns_a_eliminar = ["speaker_name", "speaker_name_clean"]
df_final = df_diputados.drop(columns=columns_a_eliminar).copy()
df_final.to_parquet("df_dialogue_master_chief_pro_crack.parquet", index=False)